In [102]:
import os
import numpy
import yfinance as yf
import pandas as pd
import ta


In [85]:
ticker = "AAPL"
start_date = "2020-01-01"
end_date = "2025-06-23"
output_file = f"{ticker}_ta_dataset.csv"

In [105]:
df = yf.download(ticker, start=start_date, end=end_date)
df.dropna(inplace=True)

/var/folders/gf/1tcqdbjj1q1bs8fps6lj_pbm0000gn/T/ipykernel_58614/3418823526.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2020-01-02,72.620850,72.681296,71.373226,71.627100,135480400
2020-01-03,71.914810,72.676439,71.689950,71.847110,146322800
2020-01-06,72.487846,72.526533,70.783248,71.034709,118387200
2020-01-07,72.146942,72.753823,71.926915,72.497529,108872000
2020-01-08,73.307510,73.609745,71.849533,71.849533,132079200
...,...,...,...,...,...
2025-06-13,196.449997,200.369995,195.699997,199.729996,51447300
2025-06-16,198.419998,198.690002,196.559998,197.300003,43020700


In [87]:
df.columns = [col[0] if isinstance(col, tuple) else col for col in df.columns]

In [88]:
df['rsi'] = ta.momentum.RSIIndicator(close=df['Close']).rsi()
df['stoch_k'] = ta.momentum.StochasticOscillator(high=df['High'], low=df['Low'], close=df['Close']).stoch()

In [89]:
macd = ta.trend.MACD(close=df['Close'])
df['macd'] = macd.macd()
df['macd_signal'] = macd.macd_signal()

df['sma_20'] = ta.trend.SMAIndicator(close=df['Close'], window=20).sma_indicator()
df['ema_20'] = ta.trend.EMAIndicator(close=df['Close'], window=20).ema_indicator()


In [90]:
bb = ta.volatility.BollingerBands(close=df['Close'])
df['bb_upper'] = bb.bollinger_hband()
df['bb_lower'] = bb.bollinger_lband()

df['atr'] = ta.volatility.AverageTrueRange(high=df['High'], low=df['Low'], close=df['Close']).average_true_range()
df['adx'] = ta.trend.ADXIndicator(high=df['High'], low=df['Low'], close=df['Close']).adx()


In [91]:
df['target'] = (df['Close'].shift(-1) > df['Close']).astype(int)

In [92]:
df.dropna(inplace=True)
df.reset_index(inplace=True)

In [93]:
df

,Date,Close,High,Low,Open,Volume,rsi,stoch_k,macd,macd_signal,sma_20,ema_20,bb_upper,bb_lower,atr,adx,target
0,2020-02-20,77.628288,78.682561,77.121754,78.192994,100566000,53.041382,73.090178,0.843613,0.950528,77.421349,77.344102,80.042917,74.799782,1.628811,13.975331,0
1,2020-02-21,75.871170,77.664649,75.253152,77.221124,129554000,46.139544,44.897085,0.625894,0.885601,77.355619,77.203823,80.062065,74.649173,1.684717,14.026662,0
2,2020-02-24,72.267258,73.721425,70.098129,72.044290,222195200,35.838933,23.558792,0.160693,0.740620,77.120815,76.733674,80.621075,73.620554,1.976741,15.870779,0
3,2020-02-25,69.819420,73.321543,69.346820,72.938615,230673600,30.808022,4.745643,-0.400882,0.512319,76.876775,76.075174,81.513996,72.239553,2.119454,17.773211,1
4,2020-02-26,70.927002,72.194554,69.436482,69.443753,198054800,35.237888,15.867500,-0.747939,0.260268,76.582454,75.584871,81.896255,71.268652,2.165069,19.539754,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1336,2025-06-13,196.449997,200.369995,195.699997,199.729996,51447300,40.802560,7.115744,-1.571946,-1.231530,201.897000,201.585117,209.242207,194.551794,5.037702,16.881351,1
1337,2025-06-16,198.419998,198.690002,196.559998,197.300003,43020700,44.244670,25.806442,-1.632476,-1.311719,201.255000,201.283677,207.353211,195.156789,4.837866,17.666974,0
1338,2025-06-17,195.639999,198.389999,195.210007,197.199997,38856200,40.652387,3.898393,-1.883062,-1.425988,200.598000,200.746184,206.115437,195.080563,4.721589,18.653920,1
1339,2025-06-18,196.580002,197.570007,195.070007,195.940002,45394700,42.356616,13.518306,-1.982945,-1.537379,200.084000,200.349405,205.061114,195.106887,4.562904,19.597532,1


In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1341 entries, 0 to 1340
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         1341 non-null   datetime64[ns]
 1   Close        1341 non-null   float64       
 2   High         1341 non-null   float64       
 3   Low          1341 non-null   float64       
 4   Open         1341 non-null   float64       
 5   Volume       1341 non-null   int64         
 6   rsi          1341 non-null   float64       
 7   stoch_k      1341 non-null   float64       
 8   macd         1341 non-null   float64       
 9   macd_signal  1341 non-null   float64       
 10  sma_20       1341 non-null   float64       
 11  ema_20       1341 non-null   float64       
 12  bb_upper     1341 non-null   float64       
 13  bb_lower     1341 non-null   float64       
 14  atr          1341 non-null   float64       
 15  adx          1341 non-null   float64       
 16  target

In [95]:
df.to_csv(output_file, index=False)
print(f"✅ Saved to {output_file}")

✅ Saved to AAPL_ta_dataset.csv


In [96]:
import feedparser
import pandas as pd
from datetime import datetime

In [ ]:
import os
import pandas as pd
from datetime import datetime
from yahoo_fin import stock_info as si

ticker = "AAPL"
csv_file = "AAPL_news_history.csv"

def get_today_news(ticker):
    try:
        news_data = si.get_news(ticker)
        today = datetime.today().date()
        headlines = []

        for item in news_data:
            try:
                title = item['title']
                link = item['link']
                published_time = pd.to_datetime(item['providerPublishTime'], unit='s').date()

                # Only keep news from today
                if published_time == today:
                    headlines.append({
                        'Date': today,
                        'Title': title,
                        'Link': link
                    })
            except Exception as e:
                print(f"Skipping a headline due to error: {e}")

        return pd.DataFrame(headlines)
    except Exception as e:
        print(f"Error fetching news for {ticker}: {e}")
        return pd.DataFrame()

def update_news_csv(ticker, csv_file):
    today_news = get_today_news(ticker)

    if today_news.empty:
        print("⚠️ No new headlines for today.")
        return

    if os.path.exists(csv_file):
        existing_data = pd.read_csv(csv_file)
        combined = pd.concat([existing_data, today_news], ignore_index=True)

        # Drop duplicates by Title + Date
        combined.drop_duplicates(subset=['Date', 'Title'], inplace=True)
        combined.sort_values(by=['Date'], inplace=True)
    else:
        combined = today_news

    combined.to_csv(csv_file, index=False)
    print(f"✅ Saved news headlines to {csv_file}")

if __name__ == "__main__":
    update_news_csv(ticker, csv_file)


In [98]:
tickers = ['AAPL']

In [99]:
news_df = pd.concat([get_yahoo_rss_headlines(t) for t in tickers])
print(news_df.head())

  ticker        date                                           headline
0   AAPL  2025-06-24  Cloud AI Today - Botpress Secures $25M To Enha...
1   AAPL  2025-06-24            Apple's Quiet AI Gambit Raises Eyebrows
2   AAPL  2025-06-24              Apple's EU Talks Enter Critical Phase
3   AAPL  2025-06-24  5 Dividend Stocks Poised to Profit From the AI...
4   AAPL  2025-06-24  US Finalizing $500 Million for African Critica...


In [100]:
news_df

,ticker,date,headline
0,AAPL,2025-06-24,Cloud AI Today - Botpress Secures $25M To Enha...
1,AAPL,2025-06-24,Apple's Quiet AI Gambit Raises Eyebrows
2,AAPL,2025-06-24,Apple's EU Talks Enter Critical Phase
3,AAPL,2025-06-24,5 Dividend Stocks Poised to Profit From the AI...
4,AAPL,2025-06-24,US Finalizing $500 Million for African Critica...
5,AAPL,2025-06-24,Google faces UK push to loosen its grip on search
6,AAPL,2025-06-24,Steve Jobs Believed Teamwork Required 'Bumping...
7,AAPL,2025-06-23,Court filings reveal OpenAI and io’s early wor...
8,AAPL,2025-06-23,Is Apple Looking To Catch Up In AI With Big Deal?
9,AAPL,2025-06-23,Apple’s Liquid Glass interface improves with r...


In [101]:
news_df.to_csv(f"{ticker}_news_dataset.csv", index=False)
print(f"✅ Saved to {ticker}_news_dataset.csv")

✅ Saved to AAPL_news_dataset.csv
